In [6]:
import requests
import re
from lxml import html 
import pandas as pd
import time

In [7]:
start_url = list(pd.read_csv("urls.csv",encoding = "utf-8")['URL'].values)

In [25]:
i = 1
inform = {}
for url in start_url:
    inform[i] = {}
    response = requests.get(url)
    tree = html.fromstring(response.text)
    for j in range(1,7):
        cate = tree.xpath('//div[@class="travel-search-sidebar-list"][{}]/h5/text()'.format(j))
        cate = [re.sub("\n\t\t","",s) for s in cate]
        keys = tree.xpath('//div[@class="travel-search-sidebar-list"][{}]/ul/li/a/text()'.format(j))
        keys = [re.sub('\n\t\t', '', s) for s in keys]
        keys = [re.sub('\n\t', '', s) for s in keys]
        keys = [s for s in keys if s!='']
        values = tree.xpath('//div[@class="travel-search-sidebar-list"][{}]/ul/li/a/span/text()'.format(j))
        values = [re.sub('\D', '',s) for s in values]
        values = [int(s) for s in values]
        try:
            inform[i][cate[0]] = list(zip(keys, np.asarray(values)))
        except:
            pass
    #print(inform[i])
    i += 1

In [29]:
import numpy as np
np.save('../data/block_f.npy', inform) 
#read_dictionary = np.load('../data/block_f.npy').item()

In [30]:
block_f = pd.DataFrame.from_dict(inform).T
block_f['blockid'] = start_url
block_f.to_csv("../data/block_f.csv",index=False,sep=',')

In [3]:
def parse_page(url):
    response = requests.get(url)
    tree = html.fromstring(response.text)
    item = {}
    
    item['name'] = tree.xpath('//div[@class="location-hero-details"]/div/h1/text()')[0]
    item['location'] = tree.xpath('//div[@class="location-hero-details"]/div/p/a/text()')
    try:
        item['location_features'] = tree.xpath('//div[@class="location-features"]/ul[@class="list-unstyled"]/li/h5/text()')
    except:
        pass
    try:
        item['price'] = tree.xpath('//strong[@class="from-price"]/text()')[0]
    except:
        item['price'] = tree.xpath('//strong[@class="from-price"]/text()')
    item['petaway_features'] = tree.xpath('//h5[@class="media-heading"]/text()')
    item['description'] = ' '.join(tree.xpath('//div[@class="location-description col-sm-12"]/p/span/text()'))
    item['bedroom&suites'] = tree.xpath('//div[@class="travel-location-intro"]/text()')
    item['dining'] = tree.xpath('//div[@class="travel-location-dining travel-location-row double-border-row"]/div/div/p/text()')
    
    df = pd.DataFrame.from_dict(item, orient='index')
    df = df.transpose()
    return df
    

In [5]:
i = 1
for url in start_url:
    print('Start crawl ', url )
    df = pd.DataFrame()
    next_page = [url]
    
    while next_page != []:
        response = requests.get(next_page[0]) 
        tree = html.fromstring(response.text)
    
        location_urls = tree.xpath('//h2[@class="item-title"]/a/@href')
    
        for location_url in location_urls:
            #print(location_url)
            new_df = parse_page(location_url)
            df = pd.concat([df, new_df])
            
        next_page = tree.xpath('//li[@class="next"]/a/@href')
        
        time.sleep(3)
        print('next page')
    path = str(i)+'.csv'
    i+=1
    df.to_csv(path)

Start crawl  https://www.petspyjamas.com/travel/collections/new-in
next page
next page
next page
next page
next page
next page
next page
next page
next page
Start crawl  https://www.petspyjamas.com/travel/collections/unique-stays
next page
next page
next page
next page
Start crawl  https://www.petspyjamas.com/travel/collections/boutique-chic 
next page
next page
Start crawl  https://www.petspyjamas.com/travel/collections/Summer-2018 
next page
next page
next page
next page
next page
next page
next page
next page
next page
Start crawl  https://www.petspyjamas.com/travel/collections/Luxury-Dog-Friendly-Cottages-And-Country-Houses
next page
next page
next page
Start crawl  https://www.petspyjamas.com/travel/collections/great-for-groups
next page
next page
next page
next page
Start crawl  https://www.petspyjamas.com/travel/collections/luxury-escapes
next page
next page
next page
Start crawl  https://www.petspyjamas.com/travel/nav/dog-friendly-dine-with-pet
next page
next page
next page
nex